### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
print(movies.shape)
print(reviews.shape)

(33725, 35)
(791152, 5)


In [3]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,News,Family,Music,...,Short,Adventure,Film-Noir,Drama,Adult,Biography,Crime,Sci-Fi,Romance,Game-Show
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,111161,10,1373234211,2013-07-07 21:56:51
1,1,117060,7,1373415231,2013-07-10 00:13:51
2,1,120755,6,1373424360,2013-07-10 02:46:00
3,1,317919,6,1373495763,2013-07-10 22:36:03
4,1,454876,10,1373621125,2013-07-12 09:25:25


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [11]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,111161,10,1373234211,2013-07-07 21:56:51
1,1,117060,7,1373415231,2013-07-10 00:13:51
2,1,120755,6,1373424360,2013-07-10 02:46:00
3,1,317919,6,1373495763,2013-07-10 22:36:03
4,1,454876,10,1373621125,2013-07-12 09:25:25


In [20]:
movie_ratings = reviews.groupby('movie_id')['rating']
movie_ratings.head()

0         10
1          7
2          6
3          6
4         10
5          8
6          8
7          9
8          5
9          8
10         6
11         7
12         9
13         3
14         8
15         8
16         7
17         1
18        10
19         8
20         8
21         7
22         8
23         7
24         7
25         7
26         8
27         8
28         7
29         8
          ..
790002     5
790010     2
790067     7
790080     7
790083     6
790095     9
790113    10
790114     9
790190     8
790220    10
790228     6
790236     6
790287    10
790296     9
790335     2
790366    10
790435     1
790439     7
790442     6
790444     1
790445     8
790446     1
790578     4
790971     6
791002     6
791059     5
791064     5
791071     4
791105     1
791125     5
Name: rating, Length: 91647, dtype: int64

In [19]:
avg_ratings = movie_ratings.mean()
avg_ratings.head()

,user_id,rating,timestamp
movie_id,,,
8,43623.000000,5.0,1.396981e+09
10,52489.000000,10.0,1.412879e+09
12,41936.000000,10.0,1.439249e+09
25,33339.000000,8.0,1.488190e+09
91,18730.666667,6.0,1.493503e+09


In [25]:
# avg_ratings
# num_ratings
# last_ratings

In [21]:
def create_ranked_df(movies, reviews):
        '''
        INPUT
        movies - the movies dataframe
        reviews - the reviews dataframe
        
        OUTPUT
        ranked_movies - a dataframe with movies that are sorted by highest avg rating, more reviews, 
                        then time, and must have more than 4 ratings
        '''
        
        # Pull the average ratings and number of ratings for each movie
        movie_ratings = reviews.groupby('movie_id')['rating']
        avg_ratings = movie_ratings.mean()
        num_ratings = movie_ratings.count()
        last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
        last_rating.columns = ['last_rating']

        # Add Dates
        rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        # merge with the movies dataset
        movie_recs = movies.set_index('movie_id').join(rating_count_df)

        # sort by top avg rating and number of ratings
        ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        # for edge cases - subset the movie list to those with only 5 or more reviews
        ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
        
        return ranked_movies

In [22]:
def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    top_movies = list(ranked_movies['movie'][:n_top])  
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [23]:
# Put your solutions for each of the cases here
ranked_movies = create_ranked_df(movies, reviews) # only run this once - it is not fast

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations('1', 20, ranked_movies)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations('53968', 5, ranked_movies)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('70000', 100, ranked_movies)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35, ranked_movies)



In [24]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [26]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    REDO THIS DOC STRING
    
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:  # multiple genres
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # create top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies


In [27]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

# Top 5 movies recommended for id 53968 with no genre filter but years=['2015', '2016', '2017', '2018']
recs_5_for_53968_filtered = popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018'])

# Top 100 movies recommended for id 70000 with no year filter but genres=['History', 'News']
recs_100_for_70000_filtered = popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News'])



In [28]:
recs_20_for_1_filtered

["Hillary's America: The Secret History of the Democratic Party (2016)",
 'Namhansanseong (2017)',
 'I Believe in Miracles (2015)',
 'O.J.: Made in America (2016)',
 'They Shall Not Grow Old (2018)',
 'Ayla: The Daughter of War (2017)',
 'Hacksaw Ridge (2016)',
 'Werk ohne Autor (2018)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Silicon Cowboys (2016)',
 '13th (2016)',
 'Ethel &amp; Ernest (2016)',
 'Paul, Apostle of Christ (2018)',
 'Kono sekai no katasumi ni (2016)',
 'Kincsem (2017)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Nise - O Coração da Loucura (2015)',
 'Under sandet (2015)']

In [29]:
recs_5_for_53968_filtered

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Five Minutes (2017)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)']

In [30]:
recs_100_for_70000_filtered

['Birlesen Gonuller (2014)',
 'Mad As Hell (2014)',
 "Hillary's America: The Secret History of the Democratic Party (2016)",
 'Seppuku (1962)',
 'Namhansanseong (2017)',
 'The Decline of Western Civilization (1981)',
 'Night Will Fall (2014)',
 "La passion de Jeanne d'Arc (1928)",
 'The Message (1977)',
 'Amadeus (1984)',
 'I Believe in Miracles (2015)',
 "Schindler's List (1993)",
 'The Culture High (2014)',
 'Ningen no jôken (1959)',
 'For Greater Glory: The True Story of Cristiada (2012)',
 'Andrey Rublyov (1966)',
 'Barry Lyndon (1975)',
 'Apollo 11 (2019)',
 'Requiem for the American Dream (2015)',
 'Braveheart (1995)',
 'O.J.: Made in America (2016)',
 'Jodhaa Akbar (2008)',
 'Lawrence of Arabia (1962)',
 'Tasogare Seibei (2002)',
 'The Mask You Live In (2015)',
 'Z (1969)',
 'Aguirre, der Zorn Gottes (1972)',
 'They Shall Not Grow Old (2018)',
 "Intolerance: Love's Struggle Throughout the Ages (1916)",
 'Mrs Brown (1997)',
 'The Salt of the Earth (2014)',
 'Russkiy kovcheg (2002